In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("csv_data/floki_1year_data.csv")

# Display the first 5 rows
print(df.head())

         date     price        volume
0  2024-04-23  0.000176  2.094265e+08
1  2024-04-24  0.000193  3.821077e+08
2  2024-04-25  0.000184  3.215056e+08
3  2024-04-26  0.000187  2.709982e+08
4  2024-04-27  0.000182  1.933730e+08


In [2]:
summary = df[['price', 'volume']].describe()
print(summary)

            price        volume
count  365.000000  3.650000e+02
mean     0.000152  3.253919e+08
std      0.000058  2.825542e+08
min      0.000047  4.638195e+07
25%      0.000118  1.663710e+08
50%      0.000147  2.409472e+08
75%      0.000182  3.635585e+08
max      0.000322  2.011276e+09


In [3]:
# Calculate percentage change in price and volume
df['price_change_pct'] = df['price'].pct_change() * 100
df['volume_change_pct'] = df['volume'].pct_change() * 100

# Display first few rows with the new columns
print(df[['date', 'price', 'volume', 'price_change_pct', 'volume_change_pct']].head(10))

         date     price        volume  price_change_pct  volume_change_pct
0  2024-04-23  0.000176  2.094265e+08               NaN                NaN
1  2024-04-24  0.000193  3.821077e+08          9.257380          82.454332
2  2024-04-25  0.000184  3.215056e+08         -4.444249         -15.859957
3  2024-04-26  0.000187  2.709982e+08          1.766629         -15.709650
4  2024-04-27  0.000182  1.933730e+08         -2.652268         -28.644170
5  2024-04-28  0.000174  1.410267e+08         -4.840775         -27.070129
6  2024-04-29  0.000170  1.197139e+08         -2.246041         -15.112600
7  2024-04-30  0.000170  1.280286e+08          0.391904           6.945455
8  2024-05-01  0.000157  1.389890e+08         -8.048720           8.560957
9  2024-05-02  0.000159  2.010175e+08          1.427969          44.628319


In [4]:
# Define default thresholds
price_threshold = 5     # percent
volume_threshold = 50   # percent

# Detect pump and dump days
df['is_pump'] = (df['price_change_pct'] > price_threshold) & (df['volume_change_pct'] > volume_threshold)
df['is_dump'] = (df['price_change_pct'] < -price_threshold) & (df['volume_change_pct'] > volume_threshold)

# Display detected days
print(df[df['is_pump'] | df['is_dump']][['date', 'price', 'volume', 'price_change_pct', 'volume_change_pct', 'is_pump', 'is_dump']])

           date     price        volume  price_change_pct  volume_change_pct  \
1    2024-04-24  0.000193  3.821077e+08          9.257380          82.454332   
12   2024-05-05  0.000192  3.490942e+08         10.099009          92.760699   
21   2024-05-14  0.000183  3.302159e+08          7.669217         367.286098   
28   2024-05-21  0.000215  2.249548e+08         11.221588          70.676117   
29   2024-05-22  0.000230  4.371814e+08          7.190202          94.341846   
35   2024-05-28  0.000277  9.605411e+08         23.294516         323.216577   
42   2024-06-04  0.000269  6.171101e+08         10.214779         144.736198   
56   2024-06-18  0.000182  3.062098e+08        -12.132972          70.139414   
63   2024-06-25  0.000168  3.481013e+08          6.418568          71.746313   
71   2024-07-03  0.000182  3.706805e+08          7.126695          52.259662   
84   2024-07-16  0.000184  5.010493e+08         20.837807         157.649984   
105  2024-08-06  0.000110  6.365906e+08 

In [ ]:
df.to_csv("csv_data_processed/floki_1year_data_with_flags.csv", index=False)


In [6]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot

# Load processed data
df = pd.read_csv("csv_data_processed/floki_1year_data_with_flags.csv", parse_dates=["date"])

# Traces
price_trace = go.Scatter(
    x=df['date'],
    y=df['price'],
    mode='lines',
    name='Price',
    line=dict(color='deepskyblue'),
    visible=True
)

volume_trace = go.Scatter(
    x=df['date'],
    y=df['volume'],
    mode='lines',
    name='Volume',
    line=dict(color='limegreen'),
    yaxis='y2',
    visible=False
)

# Pump markers (green dots)
pump_trace = go.Scatter(
    x=df[df['is_pump'] == True]['date'],
    y=df[df['is_pump'] == True]['price'],
    mode='markers',
    name='Pump',
    marker=dict(color='lime', size=10, symbol='circle'),
    visible=True
)

# Dump markers (red dots)
dump_trace = go.Scatter(
    x=df[df['is_dump'] == True]['date'],
    y=df[df['is_dump'] == True]['price'],
    mode='markers',
    name='Dump',
    marker=dict(color='red', size=10, symbol='x'),
    visible=True
)

# Layout with dual y-axes and custom styling
layout = go.Layout(
    title='Bitcoin Price and Volume with Pump/Dump Markers',
    xaxis=dict(title='Date', color='white'),
    yaxis=dict(title='Price', color='deepskyblue'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False,
        color='limegreen'
    ),
    plot_bgcolor='#0f0f23',  # Dark background
    paper_bgcolor='#0f0f23',
    font=dict(color='white'),
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.5,
            y=1.2,
            showactive=True,
            buttons=list([
                dict(label="Price Only",
                     method="update",
                     args=[{"visible": [True, False, True, True]},
                           {"title": "Bitcoin Price Only"}]),
                dict(label="Volume Only",
                     method="update",
                     args=[{"visible": [False, True, False, False]},
                           {"title": "Bitcoin Volume Only"}]),
                dict(label="Price + Volume",
                     method="update",
                     args=[{"visible": [True, True, True, True]},
                           {"title": "Bitcoin Price + Volume"}])
            ]),
        )
    ]
)

# Create figure
fig = go.Figure(data=[price_trace, volume_trace, pump_trace, dump_trace], layout=layout)
plot(fig, filename='floki_interactive_price_volume_pump_dump.html', auto_open=False)

'floki_interactive_price_volume_pump_dump.html'